In [1]:
import pandas as pd

In [2]:
chatbotData = pd.read_csv('data/ChatData.csv')
question, answer = list(chatbotData['Q']) , list(chatbotData['A'])

len(answer)

11823

In [3]:
# test로 100개만
question = question[:100]
answer = answer[:100]

for i in range(100) :
    print('질문:',question[i])
    print('답변:',answer[i])
    print(' ')

질문: 12시 땡!
답변: 하루가 또 가네요.
 
질문: 1지망 학교 떨어졌어
답변: 위로해 드립니다.
 
질문: 3박4일 놀러가고 싶다
답변: 여행은 언제나 좋죠.
 
질문: 3박4일 정도 놀러가고 싶다
답변: 여행은 언제나 좋죠.
 
질문: PPL 심하네
답변: 눈살이 찌푸려지죠.
 
질문: SD카드 망가졌어
답변: 다시 새로 사는 게 마음 편해요.
 
질문: SD카드 안돼
답변: 다시 새로 사는 게 마음 편해요.
 
질문: SNS 맞팔 왜 안하지ㅠㅠ
답변: 잘 모르고 있을 수도 있어요.
 
질문: SNS 시간낭비인 거 아는데 매일 하는 중
답변: 시간을 정하고 해보세요.
 
질문: SNS 시간낭비인데 자꾸 보게됨
답변: 시간을 정하고 해보세요.
 
질문: SNS보면 나만 빼고 다 행복해보여
답변: 자랑하는 자리니까요.
 
질문: 가끔 궁금해
답변: 그 사람도 그럴 거예요.
 
질문: 가끔 뭐하는지 궁금해
답변: 그 사람도 그럴 거예요.
 
질문: 가끔은 혼자인게 좋다
답변: 혼자를 즐기세요.
 
질문: 가난한 자의 설움
답변: 돈은 다시 들어올 거예요.
 
질문: 가만 있어도 땀난다
답변: 땀을 식혀주세요.
 
질문: 가상화폐 쫄딱 망함
답변: 어서 잊고 새출발 하세요.
 
질문: 가스불 켜고 나갔어
답변: 빨리 집에 돌아가서 끄고 나오세요.
 
질문: 가스불 켜놓고 나온거 같아
답변: 빨리 집에 돌아가서 끄고 나오세요.
 
질문: 가스비 너무 많이 나왔다.
답변: 다음 달에는 더 절약해봐요.
 
질문: 가스비 비싼데 감기 걸리겠어
답변: 따뜻하게 사세요!
 
질문: 가스비 장난 아님
답변: 다음 달에는 더 절약해봐요.
 
질문: 가장 확실한 건 뭘까?
답변: 가장 확실한 시간은 오늘이에요. 어제와 내일을 놓고 고민하느라 시간을 낭비하지 마세요.
 
질문: 가족 여행 가기로 했어
답변: 온 가족이 모두 마음에 드는 곳으로 가보세요.
 
질문: 가족 여행 고고
답변: 온 가족이 모두 마음에 드는 곳으로 가보세요.
 
질문: 가족 여행 어디로 가지?
답변: 

In [4]:
import re
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from konlpy.tag import Okt
from keras import models, layers, optimizers, preprocessing, metrics

Using TensorFlow backend.


In [5]:
# 태그단어
PAD = "<PADDING>"  # 패딩
STA = "<START>"    # 시작
END = "<END>"      # 끝
OOV = "<OOV>"      # Out Of Vocabulary

PAD_INDEX  = 0
STA_INDEX  = 1
END_INDEX  = 2
OOV_INDEX  = 3

ENCODER_INPUT = 0
DECODER_INPUT = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
maxSequences = 30

# 임베딩 벡터 차원
embeddingDim = 100

# LSTM 출력 차원
lstmHiddenDim = 128

# 정규표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

In [6]:
# 형태소 분석기
def posTag(sentences) : 
    
    tagger = Okt()
    sentencePos = []
    
    for sentence in sentences :
        # 특수문자 제거
        sentence = re.sub(RE_FILTER, '', sentence)
        sentence = ' '.join(tagger.morphs(sentence))
        sentencePos.append(sentence)
        
    return sentencePos

question = posTag(question)
answer = posTag(answer)

In [7]:
# 질문 + 대답 문장을 하나로 합치기
sentences = []
sentences.extend(question)
sentences.extend(answer)
len(sentences)  # 200

# 단어배열 생성
words = []

for sentence in sentences :
    for word in sentence.split() :
        words.append(word)
len(words) # 966개

# words에서 길이가 0인 단어를 삭제
words = [w for w in words if len(w)>0]
len(words)  # 966개

# 중복단어 삭제
words = list(set(words))
len(words)  # 450개

# 리스트 맨 앞에 추가
words[:0] = [PAD, STA, END, OOV]

# 인덱스 부여해 딕셔너리 생성
wordToIndex = dict(zip(words, range(len(words))))
indexToWord = dict(zip(range(len(words)),words ))

# 전처리
# 문장 => 인덱스
def convertTextToIndex(sentences, voc, mytype) : 
    
    sentencesIndex = []
    
    for sentence in sentences :
        sentenceIndex = []
        if mytype == DECODER_INPUT : 
            sentenceIndex.extend([voc[STA]])
        for word in sentence.split() :
            if voc.get(word) :
                sentenceIndex.extend([voc.get(word)])
            else :
                sentenceIndex.extend([voc.get(OOV)])
                
        if mytype == DECODER_TARGET : 
            # 디코더 출력은 맨 마지막에 <END>가 추가
            if maxSequences <= len(sentenceIndex) :
                sentenceIndex = sentenceIndex[:maxSequences-1] + [voc[END]]
            else :
                sentenceIndex += [voc[END]]
        
        elif len(sentenceIndex) > maxSequences :
            sentenceIndex = sentenceIndex[:maxSequences]
        # 패딩
        sentenceIndex += [wordToIndex[PAD]] * (maxSequences-len(sentenceIndex))
        
        # 결과값 저장
        sentencesIndex.append(sentenceIndex)
    
    return np.asarray(sentencesIndex)

# 인코더 입력, 디코더 입력 출력 -> 인덱스 변환
xEncoder = convertTextToIndex(question, wordToIndex, ENCODER_INPUT)
xDecoder = convertTextToIndex(answer, wordToIndex, DECODER_INPUT)
yDecoder = convertTextToIndex(answer, wordToIndex, DECODER_TARGET)

In [8]:
print(xEncoder[0])  # 12시 땡
print(xDecoder[0])  # START 하루 가 또 가네요
print(yDecoder[0])  #       하루 가 또 가네요 END

[187 234   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
[  1 120 140 182 449   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]
[120 140 182 449   2   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0]


In [33]:
import MeCab
me = MeCab.Tagger()
res = me.parse('드라이브할 때 듣기 좋은 노래 들려줘')
print(res)

드라이브	NNG,*,F,드라이브,*,*,*,*
할	XSV+ETM,*,T,할,Inflect,XSV,ETM,하/XSV/*+ᆯ/ETM/*
때	NNG,*,F,때,*,*,*,*
듣	VV,*,T,듣,Inflect,VV,VV,들/VV/*
기	ETN,*,F,기,*,*,*,*
좋	VA,*,T,좋,*,*,*,*
은	ETM,*,T,은,*,*,*,*
노래	NNG,*,F,노래,*,*,*,*
들려줘	VV+EC+VX+EC,*,F,들려줘,Inflect,VV,EC,들리/VV/*+어/EC/*+주/VX/*+어/EC/*
EOS

